# IIC1005 2018-2: Ayudantía Machine Learning

La idea de esta ayudantía es guiar el preprocesamiento de datos que deben realizar para su tarea. Además, veremos reducción de dimensionalidad y clustering en un set de datos.

In [ ]:
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.cluster import DBSCAN
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from random import shuffle
import pandas as pd
import altair as alt
import numpy as np

alt.renderers.enable('notebook')
digits = load_digits()

En primer lugar cargamos datos que vienen con **sklearn**, estos son los datos MNIST, un dataset de dígitos, cuyos datos representan a una matriz de 8x8 con un dibujo de un número y una label que representa al numero mismo.

In [ ]:
data, targets = (pd.DataFrame(digits['data']), pd.DataFrame(digits['target']))

data = pd.DataFrame(normalize(data))

data.head()

Ahora usaremos t-SNE, un algoritmo de reducción de dimensionalidad para mapear cada vector de dimensión 64 a otro de dimensión 2.

t-SNE funciona de modo que los puntos que se encuentran cercanos en el espacio de 64 dimensiones también lo esten en 2 dimensiones y lo mismo para puntos que se encuentran lejanos.

En el ejemplo se agrega también una letra, sin embargo esto sólo es para que altair nos muestre cada punto distinto con un color correspondiente a la label, lo que hace que todos los números iguales se muestren de un mismo color.

In [ ]:
def color(num):
    return chr(num + 66)

ts = TSNE(n_components=2).fit_transform(data)
tsne = pd.DataFrame(ts, columns=['x', 'y'])
tsne['label'] = targets.applymap(lambda x: color(x))
tsne.head(15)

Ahora que poseemos vectores de sólo dos dimensiones, podemos usar Altair para plotearlos.

In [ ]:
alt.Chart(tsne).mark_point().encode(
    x='x',
    y='y',
    color='label'
)

Ahora usaremos DBSCAN, un algoritmo de **clustering** para crear clusters de los puntos. Este algoritmo crea clusters a partir de la densidad de los puntos. Es decir que un conjunto de puntos cercanos formarán parte del mismo cluster, mientras que los puntos mas alejados no serán parte de él.

El parámetro que recibe DBSCAN es **eps**, que representa la distancia máxima que pueden tener dos puntos que están en el mismo cluster y **min\_samples**, que representa la cantidad de puntos que debe tener a su alrededor otro punto para ser considerado como **punto central** de un cluster.

Igual que en el ejemplo anterior, usamos letras para colorear los puntos, sin embargo estas no dependen de las labels originales de los datos, sino que de los clusters entregados por el algoritmo.

In [ ]:
clustering = DBSCAN(eps=7, min_samples=5).fit(ts)

cluster_labels = pd.DataFrame(data=clustering.labels_)

dbscan = pd.DataFrame(ts, columns=['x', 'y'])

dbscan['label'] = cluster_labels.applymap(lambda x: color(x))

dbscan.head()

In [ ]:
alt.Chart(dbscan).mark_point().encode(
    x='x',
    y='y',
    color='label'
)

Veamos que pasa si bajamos **eps** de 7 a 4

In [ ]:
clustering = DBSCAN(eps=4, min_samples=5).fit(ts)

cluster_labels = pd.DataFrame(data=clustering.labels_)

dbscan = pd.DataFrame(ts, columns=['x', 'y'])

dbscan['label'] = cluster_labels.applymap(lambda x: color(x))

dbscan.head()

alt.Chart(dbscan).mark_point().encode(
    x='x',
    y='y',
    color='label'
)

Ahora, volveremos a nuestros datos originales de dimensión 64 para entrenar clasificadores.

En primer lugar, separaremos los datos 70%/30%. Usualmente mezclamos los datos, pero esta vez no lo haremos.

In [ ]:
amount = round(len(digits['data']) * 0.7)
training_data, training_labels = (digits['data'][:amount], digits['target'][:amount])
test_data, test_labels = (digits['data'][amount:], digits['target'][amount:])

Entrenaremos un modelo que usa **SGD** o **Stochastic Gradiant Descent** para clasificar por medio de la minimización del error que produce el mismo modelo.

Es importante que para entrenar el modelo **sólo usemos los datos de entrenamiento**. La idea es que el clasificador no pueda ver los datos con los que lo probaremos antes.

In [ ]:
sgd = SGDClassifier(max_iter=1000)

sgd.fit(training_data, training_labels)

print('Score modelo con datos de entrenamiento: {}\n'.format(sgd.score(training_data, training_labels)))

print('Score modelo con datos de test: {}\n'.format(sgd.score(test_data, test_labels)))

El número que nos entrega el método score nos dice que tan bueno es nuestro modelo para predecir. Como pueden ver, el score es muy cercano a 1 cuando lo probamos con datos de entrenamiento, esto es porque el modelo "ya conoce" los datos de entrenamiento, por lo que es importante probarlo con datos que no haya visto antes. Es por esto que para el score usamos los datos de testing que separamos anteriormente.

Ahora veremos una **matriz de confusión**, la cual nos permite visualizar los resultados del modelo.

En una matriz de confusión, el eje X corresponde a la clase verdadera de un dato, mientras que el eje Y corresponde a la clase predecida por el modelo.

In [ ]:
test_predict = sgd.predict(test_data)

pd.DataFrame(confusion_matrix(test_labels, test_predict))

Podemos usar Altair para visualizar mejor la matriz, y vemos que la diagonal posee colores mas claros, significando un mayor valor en ese elemento.

La diagonal corresponde a todos los valores predichos que tienen el mismo valor real. Es decir que estan clasificados correctamente.

In [ ]:
x, y = np.meshgrid(range(0, 10), range(0, 10))
z = confusion_matrix(test_labels, test_predict)

data = pd.DataFrame({'x': x.ravel(),
                     'y': y.ravel(),
                     'z': z.ravel()})

alt.Chart(data).mark_rect().encode(
    x='x:O',
    y='y:O',
    color='z:Q'
)

Ahora, clasificaremos por medio de una regresión logística multiclase, la cual usa múltiples regresiones logísticas para clasificar cada clase.

Una regresión normal separa los datos en dos, por lo que de deben hacer varias para poder clasificar múltiples clases.

Este modelo recibe varios parámetros, sin embargo usaremos el valor de **tol** o **tolerancia**, el cual mide la tolerancia del modelo frente a los errores.

Por defecto toma un valor de 10^4.

In [ ]:
logistic_n = LogisticRegression()

logistic_n.fit(training_data, training_labels)

logistic_n.score(test_data, test_labels)

Entrenamos un segundo modelo, con mucho mayor tolerancia a los errores.

Podemos ver, al entrenarlo, que el score es bastante menor. Esto es debido a que, al llegar a un valor tolerable, el modelo deja de mejorar y termina. Como la tolerancia es mayor, el resultado no es tan bueno.

In [ ]:
logistic = LogisticRegression(tol=3)

logistic.fit(training_data, training_labels)

logistic.score(test_data, test_labels)

Ahora veremos las matrices de confusión de cada modelo.

In [ ]:
test_predict_1 = logistic_n.predict(test_data)

pd.DataFrame(confusion_matrix(test_labels, test_predict_1))

In [ ]:
test_predict_2 = logistic.predict(test_data)

pd.DataFrame(confusion_matrix(test_labels, test_predict_2))

Podemos observar que el segundo modelo tiene una matriz de confusión mucho menos diagonal, significando muchos errores de clasificación.

Para visualizar mejor, nuevamente mostraremos las matrices por medio de altair. Podemos observar de mejor manera la diferencia de los modelos frente a un cambio de parámetros.

In [ ]:
x, y = np.meshgrid(range(0, 10), range(0, 10))
z_1 = confusion_matrix(test_labels, test_predict_1)
z_2 = confusion_matrix(test_labels, test_predict_2)

data_1 = pd.DataFrame({'x': x.ravel(),
                       'y': y.ravel(),
                       'z': z_1.ravel(),
                       'd': 'tol = 1e-4'})

data_2 = pd.DataFrame({'x': x.ravel(),
                       'y': y.ravel(),
                       'z': z_2.ravel(),
                       'd': 'tol = 3'})

total_data = pd.concat([data_1, data_2])

alt.Chart(total_data).mark_rect().encode(
    column='d',
    x='x:O',
    y='y:O',
    color='z:Q'
)

Finalmente, clasificaremos usando un modelo de KNN, el cual dado un vector, busca los K vectores mas cercanos y toma la desición dependiendo de la mayor cantidad de vecinos iguales.

In [ ]:
knn_1 = KNeighborsClassifier(n_neighbors=5)

knn_1.fit(training_data, training_labels)

knn_1.score(test_data, test_labels)

In [ ]:
knn_1_predict = knn_1.predict(test_data)

pd.DataFrame(confusion_matrix(test_labels, knn_1_predict))

In [ ]:
x, y = np.meshgrid(range(0, 10), range(0, 10))
z_1 = confusion_matrix(test_labels, knn_1_predict)

data = pd.DataFrame({'x': x.ravel(),
                       'y': y.ravel(),
                       'z': z_1.ravel()})

alt.Chart(data).mark_rect().encode(
    x='x:O',
    y='y:O',
    color='z:Q'
)

In [ ]:
knn_2 = KNeighborsClassifier(n_neighbors=100)

knn_2.fit(training_data, training_labels)

knn_2_predict = knn_2.predict(test_data)

knn_3 = KNeighborsClassifier(n_neighbors=500)

knn_3.fit(training_data, training_labels)

knn_3_predict = knn_3.predict(test_data)

print()

In [ ]:
x, y = np.meshgrid(range(0, 10), range(0, 10))
z_2 = confusion_matrix(test_labels, knn_2_predict)
z_3 = confusion_matrix(test_labels, knn_3_predict)

data_1 = pd.DataFrame({'x': x.ravel(),
                       'y': y.ravel(),
                       'z': z_1.ravel(),
                       'd': 'k = 5'})

data_2 = pd.DataFrame({'x': x.ravel(),
                       'y': y.ravel(),
                       'z': z_2.ravel(),
                       'd': 'k = 20'})

data_3 = pd.DataFrame({'x': x.ravel(),
                       'y': y.ravel(),
                       'z': z_3.ravel(),
                       'd': 'k = 500'})

total_data = pd.concat([data_1, data_2, data_3])

alt.Chart(total_data).mark_rect().encode(
    column='d',
    x='x:O',
    y='y:O',
    color='z:Q'
)